# OpenParticle

The main classes of this 'package' are `FockState` and `ParticleOperator`. Their functionality is as follows.

In [37]:
#Imports
from FockState import *
from ParticleOperator import *
from qubit_mappings import *
import symmer

## `FockState` Class

A Fock state in second quantization is a quantum state given by an occupancy representation of some particular mode(s). In quantum field theory, we work mainly in momentum space, so these modes represent the occupied momentum of some particle. For example, for a particle with potential momentum modes $p \in \{0, 1, 2, 3, \dots, N\}$, the corresponding Fock state vector for a particle in momentum mode $2$ would be $|0, 0, 1, 0, \dots, 0 \rangle$

In general, we are interested in hadrons, or bound states of Quantum Chromodynamics (QCD). Because of this, we need a representation of our hadronic states in terms of fermions (quarks), antifermions (antiquarks), and bosons (gluons). The way this Fock state is written is: $|f;\bar{f}; b\rangle = |f\rangle \otimes |\bar{f}\rangle\otimes |b\rangle$, where each particle type has its own representation as shown above. Fermions and antifermions obey fermi statistics such that the occupancy of fermions in a particular mode, $n^{f, \bar{f}}_i \in \{0, 1\}$, while gluons obey bose statistics with occupancies in a given mode, $n^b_i \in \{0, 1, 2, 3, \cdots \}$

To define your Fock state, you need to pass in a vector corresponding to occupied fermion modes, occupied antifermion modes, and occupied bosonic modes designated how many bosons exist in each mode. The four parameters to pass when instantiating the class are `ferm_occupancy`, `antiferm_occupancy`, `bos_occupancy`, and `coeff`. The first three are lists of length $N$ where each non-zero entry denotes a particle of the particular type to have the corresponding momentum orbital occupied, with the non-zero number representing the occupancy number. `coeff` is a parameter specifying the coefficient in front of the Fock state. 

In [2]:
#e.g. N = 3

f_occ = [1, 0, 0]
af_occ = [0, 0, 1]
b_occ = [4, 0, 0]

x = FockState(ferm_occupancy = f_occ, antiferm_occupancy = af_occ,
              bos_occupancy = b_occ, coeff = 1.0)

print(x)

1.0 * |1,0,0; 0,0,1; 4,0,0⟩


### `FockState` Methods:

- `compact_state()`:

We can also print a more compact encoding, which will be useful when N is very large. The `.compact_state()` method will accomplish this. For fermions and antifermions, the values in the compact encoding correspond to the occupied modes while for bosons, it returns a tuple $(i, n^b_i)$ in order to display information about how many bosons are in each mode. 

In [3]:
x.compact_state()

'|0; 2; (0, 4)⟩'

- `dagger()`: 

Returns the bra corresponding to the Fock state ket.

In [4]:
print(x.dagger())

1.0 * ⟨1,0,0; 0,0,1; 4,0,0|


### Related Classes to FockState

### `FockStateSum`

You can add Fock states the same way that you add kets. Note that you need to make sure the number of modes is the same for each state. 

In [5]:
state_1 = FockState([1], [0], [0], 1)
state_2 = FockState([0], [1], [1], 1)
state = state_1 + state_2
print(state)

1 * |1; 0; 0⟩ + 1 * |0; 1; 1⟩


### `ConjugateFockState`

`ConjugateFockState` is instantiated in two ways. The first is identical to how `FockState` is instantiated. The second is via the method `.from_state(state: FockState)`, which essentially turns a ket into a bra. 

In [6]:
y = ConjugateFockState([1, 0, 0], [0, 0, 1], [4, 0, 0], 1)
z = ConjugateFockState.from_state(x)
print(y)
print(z)

1 * ⟨1,0,0; 0,0,1; 4,0,0|
1.0 * ⟨1,0,0; 0,0,1; 4,0,0|


With the `ConjugateFockState` class defined, we can now take inner products via `*` or with `conj_state.inner_product(state)`

In [7]:
print(y * z)
print(y.inner_product(z))

1.0
1.0


## `ParticleOperator` Class

The `ParticleOperator` class allows you to define products of fermionic/bosonic creation/annihilation operators that act on Fock states to increase/decrease occupancy numbers. The products of operators that can be created come from the set $\{b_p, b_p^\dagger, d_p, d_p^\dagger, a_p, a_p^\dagger  \}$. For example, the operator $b_1^\dagger b_2 a_2$ acts on a Fock state to create a fermion with momentum $p = 1$, annihilate a fermion with momentum $p = 2$ and annihilate a boson with momentum $p = 2$.

To instantiate the a product of operators for the `ParticleOperator` class, you must specify the following parameters: `particle_str`, `modes`, `ca_string`, and `coeff`. `particle_str` is a string that specifies (from left to right) the order of types of particles in the operator. `modes` is a list of integers specifying (from left to right) the modes of each operator. `ca_string` is a string that specifies (from left to right) whether each operator is a creation or annihilation operator. Using the operator above as an example, we would instantiate this operator as follows: 

In [8]:
op = ParticleOperator('ffb', [1, 2, 2], 'caa', 1)
print(op)

1*b^†_1b_2a_2


- `.display()`:

Using a `Jupyter Notebook`, we can also print the operator with Latex via the `.display()` method:

In [9]:
op_2 = ParticleOperator('ffb', [1, 2, 2], 'caa', 1)
op_2.display()

<IPython.core.display.Latex object>

There are three child classes of `ParticleOperator`.

### `FermionOperator`, `AntifermionOperator`, and `BosonOperator`

These three classes operator similarly to the `ParticleOperator` class; however, they are just single operators by themselves. Thus, we only need to specify the mode and if it is a creation or annihilation operator. Once these operators are defined, products can be taken which return an instance of the `ParticleOperator` class. 

In [10]:
f1 = FermionOperator(2, 'c', coeff = 3)
f2 = FermionOperator(1, 'a', coeff = 2)
a1 = BosonOperator(2, 'c', coeff = 3)

(f1 * f2 * a1).display()


<IPython.core.display.Latex object>

### `ParticleOperatorSum` Class

In [11]:
op_tosum_1 = ParticleOperator('ffb', [1, 2, 0], 'cca')
op_tosum_2 = ParticleOperator('fba', [1, 1, 0], 'aaa')
(op_tosum_1 + op_tosum_2).display()

<IPython.core.display.Latex object>

## Mapping Creation/Annihilation Operators to Qubit Operators

In order to transform creation and annihilation operators to operators that act on qubits, we need to define a suitable map that respects commutation relations for the particle they represent (i.e. $[a_p, a_q^
\dagger] = \delta_{p,q}$ for bosons and $\{b_p, b_q^\dagger \} = \delta_{p,q}$). The canonical way to accomplish this is via the Jordan Wigner transformation for fermions ([McArdle](https://arxiv.org/pdf/1808.10402.pdf)), and the folliwing encoding to be referred to as the unary transformation for bosons ([Somma](https://arxiv.org/pdf/quant-ph/0512209.pdf)). While these are not the only mappings, they are the only currently available for OpenParticle.

- Jordan Wigner Encoding

The Jordan Wigner encoding mapps Fock states to qubit states via: $|f_0, f_1, \dots f_N \rangle \rightarrow |q_N, \dots, q_1, q_0 \rangle$ (note the reversed ordering! This is common notation), where $q_p = f_q \in \{0, 1\}$. The corresponding map for the creation and annihilation operators that respect fermi statistics and commutation relations are: 

\begin{align}
b_p = Z_0 \otimes \dots \otimes Z_{p - 1} \otimes \frac{1}{2}(X_p - iY_p) \otimes I_{p + 1} \dots \otimes I_N\\
b_p^\dagger = Z_0 \otimes \dots \otimes Z_{p - 1} \otimes \frac{1}{2}(X_p + iY_p) \otimes I_{p + 1} \dots \otimes I_N

\end{align}

- unary Encoding

We need an alternative mapping for bosons, not just because of the different commutation relations, but also because bosons obey bose statistics, so the occupancy numbers for a given mode can be $f_q \in \{0, 1, 2, \dots \}$. The qubit requirement to map bosonic Fock states to qubit states is greater than that of fermions in general due to these statistics. In order to encode a bosonic system, we first must not only cut off the total number of modes $N$ (just like we do for fermions), but we also must cut off the total number of bosons that can exist in each mode via $N_p^{\text{max}}$. We implicitly assume that each mode has the same cutoff so that the universal cutoff for all modes is $N^\text{max}$.

For each mode $j \in \{0, N\}$, $N^\text{max}$ qubits are needed to encode the occupation number (in a one-hot encoding scheme). For example, if in mode $j = 2$ with $N^\text{max} = 5$, there are 3 bosons that exist in this mode, the Fock state would be encoded as $|1_01_11_20_31_41_5\rangle_2$, where the 2 designates the mode and there is a 0 in position 3 (from left to right it goes 0, 1, 2, etc.) which shows there are 3 bosons in this state $j = 2$ mode. To get the complete state of all $N$ modes, you would have to tensor product the kets between each mode.  

The mapping from bosonic operator to qubit operator is:

\begin{align}

a_p^\dagger = \sum_{n = 0}^{N^\text{max} - 1} \sqrt{n + 1}\sigma_-^{n,j}\sigma_+^{n + 1, j}\\
a_p = \sum_{n = 0}^{N^\text{max} - 1} \sqrt{n + 1}\sigma_+^{n,j}\sigma_-^{n + 1, j}

\end{align}

### `jordan_wigner()`, `unary()`, and `qubit_op_mapping()` Functions

- `jordan_wigner()`:

The `jordan_wigner()` function takes in as a parameter, a single fermionic creation or annihilation operator, as well as a total number of fermionic modes $N$, and returns the qubit state:

In [12]:
op = FermionOperator(2, 'c')
qubit_op = jordan_wigner(op, 3)
op.display()
print("gets mapped to:")
print(qubit_op)

<IPython.core.display.Latex object>

gets mapped to:
 0.500+0.000j ZZX +
 0.000-0.500j ZZY


- `unary()`:

The `unary()` function takes in the same parameters, but also takes in the parameter `max_bose_mode_occ` which is $N^\text{max}$.

In [13]:
bos_op = BosonOperator(3, 'a')
qubit_bos_op = unary(bos_op, 2, 3)
bos_op.display()
print("gets mapped to:")
print(qubit_bos_op)

<IPython.core.display.Latex object>

gets mapped to:
 0.250+0.000j IIIIIIIIIXXI +
 0.000-0.250j IIIIIIIIIXYI +
 0.000+0.250j IIIIIIIIIYXI +
 0.250+0.000j IIIIIIIIIYYI +
 0.354+0.000j IIIIIIIIIIXX +
 0.000-0.354j IIIIIIIIIIXY +
 0.000+0.354j IIIIIIIIIIYX +
 0.354+0.000j IIIIIIIIIIYY


In [14]:
op = ParticleOperator('fab', [1, 2, 0], 'cac')
qubit_op_mapping(op, [2, 2, 2], 3)

 0.108-0.000j ZXZZXIIXXIIIIIIII +
 0.000+0.108j ZXZZXIIXYIIIIIIII +
 0.000-0.108j ZXZZXIIYXIIIIIIII +
 0.108-0.000j ZXZZXIIYYIIIIIIII +
 0.000+0.108j ZXZZYIIXXIIIIIIII +
-0.108+0.000j ZXZZYIIXYIIIIIIII +
 0.108-0.000j ZXZZYIIYXIIIIIIII +
 0.000+0.108j ZXZZYIIYYIIIIIIII +
 0.000-0.108j ZYZZXIIXXIIIIIIII +
 0.108-0.000j ZYZZXIIXYIIIIIIII +
-0.108-0.000j ZYZZXIIYXIIIIIIII +
 0.000-0.108j ZYZZXIIYYIIIIIIII +
 0.108-0.000j ZYZZYIIXXIIIIIIII +
 0.000+0.108j ZYZZYIIXYIIIIIIII +
 0.000-0.108j ZYZZYIIYXIIIIIIII +
 0.108-0.000j ZYZZYIIYYIIIIIIII +
 0.088-0.000j ZXZZXIXXIIIIIIIII +
 0.000+0.088j ZXZZXIXYIIIIIIIII +
 0.000-0.088j ZXZZXIYXIIIIIIIII +
 0.088-0.000j ZXZZXIYYIIIIIIIII +
 0.000+0.088j ZXZZYIXXIIIIIIIII +
-0.088+0.000j ZXZZYIXYIIIIIIIII +
 0.088-0.000j ZXZZYIYXIIIIIIIII +
 0.000+0.088j ZXZZYIYYIIIIIIIII +
 0.000-0.088j ZYZZXIXXIIIIIIIII +
 0.088-0.000j ZYZZXIXYIIIIIIIII +
-0.088-0.000j ZYZZXIYXIIIIIIIII +
 0.000-0.088j ZYZZXIYYIIIIIIIII +
 0.088-0.000j ZYZZYIXXIIIIIIIII +
 0.000+0.088j 

- `qubit_state_mapping()`:

Lastly, Fock states can be mapped to qubit states with the Jordan Wigner and unary encodings above with: 

In [51]:
x = FockState([1, 0], [0,1], [2, 1], 1)
qubit_state_mapping(x, 3)

 1.000+0.000j |011011011011>